In [1]:
import pandas as pd
import geopandas as gpd
import datetime

spn 5246 spn-fmi 1569-31 faultid = recordid

In [2]:
faults = pd.read_csv('../data/J1939Faults.csv')
vehicleDiagnostic = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')
faultCodes = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')

C:\Users\hcgre\AppData\Local\Temp\ipykernel_8504\985816224.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv('../data/J1939Faults.csv')
C:\Users\hcgre\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


**Finding trucks that are within one mile of a service center** 

In [3]:
gdf = gpd.GeoDataFrame(faults, geometry=gpd.points_from_xy(faults.Longitude, faults.Latitude), crs="EPSG:4326")

In [4]:
points = pd.DataFrame({
    'lat' : [36.0666667, 35.5883333, 36.1950],
    'lon' : [-86.4347222, -86.4438888, -83.174722]
})

points = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy(points.lon, points.lat), crs="EPSG:4326")

points['buffergeometry1mile'] = points['geometry'].buffer(0.01)

points.set_geometry('buffergeometry1mile',inplace=True)

C:\Users\hcgre\AppData\Local\Temp\ipykernel_8504\4116657202.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points['buffergeometry1mile'] = points['geometry'].buffer(0.01)


In [5]:
intersection = gpd.overlay(gdf, points, how='intersection', keep_geom_type=False)

lessThanMile = intersection['RecordID'].unique().tolist()

**Filtering out faults that occurred within one mile of the service center**

In [6]:
faults = faults.loc[~faults['RecordID'].isin(lessThanMile)].copy()

faults = faults.drop(columns = 'geometry')

**Filtering the equipment id to be less than 5 in length**

In [7]:
faults['EquipmentID'] = faults['EquipmentID'].astype(str)

faults = faults.loc[faults['EquipmentID'].str.len() < 5]

**Grabbing a list of trucks that have has either a 5246 or a 1569-31**

In [8]:
trucks5246 = faults.loc[faults['spn'] == 5246]['EquipmentID'].unique().tolist()

trucks1569 = faults.loc[(faults['spn'] == 1569) & (faults['fmi'] == 31)]['EquipmentID'].unique().tolist()

faults['EventTimeStamp'] = pd.to_datetime(faults['EventTimeStamp']) #turning EventTimeStamp into a datetime

In [9]:
df = []

for truck in range(0, len(trucks5246)):
    dates = (
        faults
        .loc[
            (faults['EquipmentID'] == trucks5246[truck]) & 
            (faults['spn'] == 5246) 
            #(faults['active'] == True)
        ]['EventTimeStamp']
        .tolist()
    )
    
    for i in range(0, len(dates)):
        date = dates[i] - datetime.timedelta(days=8)
        
        df.append(
            faults.loc[
                (faults['EquipmentID'] == trucks5246[truck]) & 
                (faults['EventTimeStamp'] <= dates[i]) & 
                (faults['EventTimeStamp'] >= date) 
                #(faults['active'] == True)
            ]
        )
        
weekOf5246 = pd.concat(df).drop_duplicates('RecordID')

In [22]:
test = pd.merge(
    left = weekOf5246[['RecordID', 'spn', 'fmi', 'active']].groupby(['spn', 'fmi', 'active']).count().reset_index().sort_values(by = 'RecordID', ascending = False).rename(columns = {'RecordID' : 'count5246'}),
    right = faults.loc[~faults['EquipmentID'].isin(trucks5246)][['RecordID', 'spn', 'fmi', 'active']].groupby(['spn', 'fmi', 'active']).count().reset_index().sort_values(by = 'RecordID', ascending = False).rename(columns = {'RecordID' : 'countNo5246'}),
    how = 'outer',
    on = ['spn', 'fmi', 'active']).fillna(0)

In [24]:
test['totals'] = test['count5246'] + test['countNo5246']

In [28]:
test['percent5246'] = test['count5246'] / test['totals']

test['percentNo5246'] = test['countNo5246'] / test['totals']

In [39]:
test.loc[test['totals'] > 100].sort_values(by = 'count5246', ascending = False).head(25)

,spn,fmi,active,count5246,countNo5246,totals,percent5246,percentNo5246
0,96,3,True,1470.0,25624.0,27094.0,0.054256,0.945744
1,111,17,True,1248.0,75943.0,77191.0,0.016168,0.983832
2,111,17,False,1234.0,74638.0,75872.0,0.016264,0.983736
3,829,3,True,1082.0,26728.0,27810.0,0.038907,0.961093
4,1569,31,True,701.0,2956.0,3657.0,0.191687,0.808313
5,1569,31,False,534.0,2714.0,3248.0,0.164409,0.835591
6,5246,0,True,327.0,0.0,327.0,1.000000,0.000000
7,5246,0,False,291.0,0.0,291.0,1.000000,0.000000
8,929,9,True,140.0,98306.0,98446.0,0.001422,0.998578
9,929,9,False,140.0,95391.0,95531.0,0.001465,0.998535


In [50]:
faultCodes.loc[(faultCodes['SPN'] == 3362) & (faultCodes['J1939 FMI'] == 31)]

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
1078,Y,1682,167,Not Mapped,Not Mapped,Not Mapped,11,3362,31,Not Mapped,Amber,Warning,Aftertreatment 1 Diesel Exhaust Fluid Dosing U...,Lack of urea or air to the doser unit
